# Manipulação de Arquivos e Strings

### Semana 13 | Uso do Regex

## Questão Dirigida

Como gravar em um arquivo CSV os dados da pandemia de COVID-19 por país desta [página](https://pt.wikipedia.org/wiki/Portal:COVID-19) da Wikipedia?

## Desenvolvimento do Projeto

Para desenvolver este projeto, vamos utilizar técnicas de raspagem de dados na Web (Web Scraping), com a biblioteca `requests`, `BeautifulSoup` e expressões regulares.

In [1]:
# download
import requests

# parsing do HTML
from bs4 import BeautifulSoup

# expressões regulares
import re

A incidência de COVID-19 por país pode ser encontrada na página da Wikipedia.

In [2]:
# URL de análise
url_pagina = 'https://pt.wikipedia.org/wiki/Portal:COVID-19'

In [3]:
# Utiliza a biblioteca requests para fazer o download da pagina HTML
pagina_html = requests.get(url_pagina)

# O Código 200 indica que o download foi bem sucedido
pagina_html

<Response [200]>

In [4]:
# O Beautiful Soup 
soup = BeautifulSoup(pagina_html.content, 'html.parser')
#print(soup.prettify())

**Inspeção da página**

Inspecionando o código-fonte da página HTML no navegador (Chrome, Firefox etc) podemos ver que a tabela de interesse contém a classe CSS: `wikitable sortable mw-collapsible no-ref`. Vamos utilizar esta classe CSS para restringir a busca pelos elementos de interesse.

In [5]:
tb_covid_pais = soup.find(class_='wikitable sortable mw-collapsible no-ref')
#tb_covid_pais

Opcionalmente, pode-se fazer o seguinte.

In [6]:
tbs_covid_pais = soup.find_all(class_='wikitable sortable mw-collapsible no-ref')
# Lembre-se que o método find_all retorna uma lista. No caso deste exemplo, 
# só existe um elemento na lista
tb_covid_pais = tbs_covid_pais[0]
# tb_covid_pais

In [7]:
# Retorna uma lista com todas as linhas da tabela
linhas_tb = tb_covid_pais.find_all('tr')
#linhas_tb

> **Atenção**: É sempre importante entender o tipo de uma variável.

In [8]:
type(linhas_tb)

bs4.element.ResultSet

In [9]:
type(linhas_tb[0])

bs4.element.Tag

**Investigando os resultados**

Primeiro país ocorre na posição 2.

In [10]:
linhas_tb[2].get_text()

'\n Estados Unidos[a]\n33\xa0448\xa0422\n\n601\xa0079\n\n–\n\n[3]\n'

Último ocorre na posição 226.

In [11]:
linhas_tb[226].get_text()

'\n Saba\n7\n\n0\n\n7\n\n[292]\n'

In [12]:
ini_tb = 2
fim_tb = 226

Brincando um pouco com esses objetos podemos ver que eles contém os dados que desejamos. No entanto, esses dados precisem ser tratados. Por exemplo: podemos ver que o resultado a seguir começa e termina com `\n` (caractere de nova linha); existe um espaço antes do nome do país; existem vários `\n` e `\xa0` (*line feed*) no meio da linha. Além disso, existem notas associadas ao nome do país e ao final da linha. Vamos cuidar de cada um desses problemas.

In [13]:
lin = linhas_tb[2].get_text()
lin

'\n Estados Unidos[a]\n33\xa0448\xa0422\n\n601\xa0079\n\n–\n\n[3]\n'

##### Removendo os `\n` e os espaços no começo e ao final da linha. 

O método `strip()` remove caracteres no começo e ao final da linha.

In [14]:
lin

'\n Estados Unidos[a]\n33\xa0448\xa0422\n\n601\xa0079\n\n–\n\n[3]\n'

In [15]:
lin.strip("\n \t")

'Estados Unidos[a]\n33\xa0448\xa0422\n\n601\xa0079\n\n–\n\n[3]'

Note que a variável `lin` não é alterada desta forma.

In [16]:
lin

'\n Estados Unidos[a]\n33\xa0448\xa0422\n\n601\xa0079\n\n–\n\n[3]\n'

Para que a variável `lin` seja alterada, é necessário utilizar a atribuição.

In [17]:
lin = lin.strip("\n ")
lin

'Estados Unidos[a]\n33\xa0448\xa0422\n\n601\xa0079\n\n–\n\n[3]'

##### Removendo os caracteres `\xa0` e `\n` no meio da string

Podemos utilizar o método `replace()` para substituir o caractere `\xa0` com a string vazia, para que o número de casos não esteja com espaços entre eles. Note que o caractere `\xa0` está em Unicode (que podem ser representados por hexadecimais), por isso necessitam do `u` antes do começo da string.

In [18]:
lin = lin.replace(u"\xa0", u"")
lin

'Estados Unidos[a]\n33448422\n\n601079\n\n–\n\n[3]'

Como desejamos criar um arquivo CSV ao final do processo, vamos substituir um ou mais `\n` e `\t` que separa cada coluna da tabela por um `\t`.

In [19]:
lin = re.sub("[\n|\t]+","\t", lin)
lin

'Estados Unidos[a]\t33448422\t601079\t–\t[3]'

##### Removendo as notas do nome do país e ao final da linha

Para remover as notas do nome do país, vamos utilizar a seguinte expressão regular: `r"\[[\d|a-z]+\]"`. A expressão pode ser lida da seguinte forma: selecione o texto de dígitos ou caracteres alfa-numéricos que estejam entre `[` e `]`. 

In [20]:
lin

'Estados Unidos[a]\t33448422\t601079\t–\t[3]'

In [21]:
lin = re.sub(r"\[[\d|a-z]+\]", "", lin)
lin

'Estados Unidos\t33448422\t601079\t–\t'

Remove-se finalmente o `\t` ao final da string.

In [22]:
# Coluna Fontes
lin = lin.rstrip("\t")
lin

'Estados Unidos\t33448422\t601079\t–'

In [23]:
lin = lin.replace("–", "0")
lin

'Estados Unidos\t33448422\t601079\t0'

### Criação de uma função de preparação da linha

Para se criar a função, vamos começar aplicando a sequência de passos para preparar a linha dos EUA e em seguida analisar se o que fizemos é suficiente para os outros casos.

In [24]:
def prep_linha_v1(linha):
    # Remove os caracteres \n no começo e final da string
    # Em seguida, remove os espaços
    linha = linha.strip("\n ")
    linha = linha.replace(u"\xa0", u"")
    linha = re.sub("[\n|\t]+","\t", linha)
    linha = re.sub(r"\[[\d|a-z]+\]", "", linha)
    linha = linha.strip("\t")
    linha = re.sub(r"–", "0", linha)
    return linha

Como esperado, a função funciona para os EUA.

In [25]:
eua = prep_linha_v1(linhas_tb[2].get_text())
eua

'Estados Unidos\t33448422\t601079\t0'

#### Preste atenção nos tipos de dados da entrada das funções.

Chamada com String.

In [26]:
type(linhas_tb[2].get_text())

str

In [27]:
prep_linha_v1(linhas_tb[2].get_text())

'Estados Unidos\t33448422\t601079\t0'

Chamada com `Tag`

In [28]:
type(linhas_tb[2])

bs4.element.Tag

In [29]:
prep_linha_v1(linhas_tb[2])

TypeError: 'NoneType' object is not callable

#### Investigando outros casos

Listando as transformações nos países.

In [30]:
for i in range(ini_tb,fim_tb+1):
    linha = linhas_tb[i].get_text()
    print(i, prep_linha_v1(linha))

2 Estados Unidos	33448422	601079	0
3 Índia	26752447	303720	23728011
4 Brasil	16720081	467706	15168330
5 França	5667324	109557	0
6 Turquia	5263697	47768	5131463
7 Rússia	5090249	122267	4702599
8 Reino Unido	4494699	127794	0
9 Itália	4220304	126221	3868332
10 Argentina	3852093	79320	3409253
11 Espanha	3687762	80049	0
12 Alemanha	3703807	89316	3498050
13 Colômbia	3432422	89297	3193406
14 Irã	2935443	80488	2494108
15 Polônia	2873527	73984	2641139
16 México	2420659	227840	1930608
17 Ucrânia	2206836	50857	2062572
18 Peru	1961087	69342	1914169
19 Chéquia	1662256	30126	1622432
20 África do Sul	1669231	56601	1563719
21 Países Baixos	1651780	17632	0
22 Indonésia	1 511 712	40 858	1 348 330
23 Canadá	1383214	25566	1326484
24 Chile	1394973	29385	1321600
25 Filipinas	1240716	21158	1167426
26 Iraque	1205522	16405	1120799
27 Romênia	1078142	30415	1040869
28 Suécia	1068473	14451	0
29 Bélgica	1063499	24968	0
30 Paquistão	924667	20930	848685
31 Portugal	850262	17026	810271
32 Israel	839508	6413	832693
33

Pela Grécia, vemos que remos ter tratar um comando HTML em uma das colunas.

In [ ]:
prep_linha_v1(linhas_tb[48].get_text())

A Ossétia do Sul também necessita ser tratada.

In [31]:
prep_linha_v1(linhas_tb[183].get_text())

'Ossétia do Sul\t3343\t60+\t3198'

In [32]:
def prep_linha_v2(linha):
    # Remove os caracteres \n no começo e final da string
    # Em seguida, remove os espaços
    linha = linha.strip("\n ")
    linha = linha.replace(u"\xa0", u"")
    linha = re.sub("[\n|\t]+","\t", linha)
    linha = re.sub(r"\[[\d|a-z]+\]", "", linha)
    linha = linha.strip("\t")
    linha = re.sub(r"–", "0", linha)
        
    # Trata a Grécia (parte por parte)
    linha = linha.replace("data-sort-value=\"−1\"", "0")
    
    # Trata a Ossétia do Sul
    linha = linha.replace("+", "")
    return linha

In [33]:
prep_linha_v2(linhas_tb[48].get_text())

'Grécia\t404163\t12122\t0'

In [35]:
prep_linha_v2(linhas_tb[183].get_text())

'Ossétia do Sul\t3343\t60\t3198'

Analisando a Indonésia, descobrimos mais uma surpresa.

In [36]:
prep_linha_v2(linhas_tb[22].get_text())

'Indonésia\t1 511 712\t40 858\t1 348 330'

Para facilitar, vamos tratar este caso direto no Pandas.

##### Preparando o texto do arquivo final

Para criarmos o texto arquivo CSV final, basta fazer uma concatenação de todas as linhas da tabela, incluindo um `\n` ao final da linha. 

In [37]:
# Esta primeira linha define o título da tabela
texto_final = "País\tCasos\tMortes\tCurados\n"
for i in range(ini_tb, fim_tb+1):
    # Preste atenção no .get_text(), que transforma a chamada em string
    linha = prep_linha_v2(linhas_tb[i].get_text())
    texto_final += linha
    if (i < fim_tb):
        texto_final += "\n"
print(texto_final)

País	Casos	Mortes	Curados
Estados Unidos	33448422	601079	0
Índia	26752447	303720	23728011
Brasil	16720081	467706	15168330
França	5667324	109557	0
Turquia	5263697	47768	5131463
Rússia	5090249	122267	4702599
Reino Unido	4494699	127794	0
Itália	4220304	126221	3868332
Argentina	3852093	79320	3409253
Espanha	3687762	80049	0
Alemanha	3703807	89316	3498050
Colômbia	3432422	89297	3193406
Irã	2935443	80488	2494108
Polônia	2873527	73984	2641139
México	2420659	227840	1930608
Ucrânia	2206836	50857	2062572
Peru	1961087	69342	1914169
Chéquia	1662256	30126	1622432
África do Sul	1669231	56601	1563719
Países Baixos	1651780	17632	0
Indonésia	1 511 712	40 858	1 348 330
Canadá	1383214	25566	1326484
Chile	1394973	29385	1321600
Filipinas	1240716	21158	1167426
Iraque	1205522	16405	1120799
Romênia	1078142	30415	1040869
Suécia	1068473	14451	0
Bélgica	1063499	24968	0
Paquistão	924667	20930	848685
Portugal	850262	17026	810271
Israel	839508	6413	832693
Hungria	804382	29728	696029
Bangladesh	802305	12660	742151
Jo

## Gravando o texto em um arquivo

Para se gravar o texto em um arquivo, basta utilizar o código a seguir.

In [38]:
arquivo = open("lista-paises.tsv", "w")
arquivo.write(texto_final)
arquivo.close()

### Lendo o arquivo CSV com o Pandas

In [39]:
import pandas as pd
df = pd.read_csv("lista-paises.tsv", sep='\t')
df

,País,Casos,Mortes,Curados
0,Estados Unidos,33448422,601079,0
1,Índia,26752447,303720,23728011
2,Brasil,16720081,467706,15168330
3,França,5667324,109557,0
4,Turquia,5263697,47768,5131463
...,...,...,...,...
220,Santo Eustáquio,20,0,20
221,Montserrat,20,1,18
222,MS Zaandam,13,4,0
223,Coral Princess,12,3,0


### Indonésia

In [40]:
df[df['País'] == 'Indonésia']

,País,Casos,Mortes,Curados
20,Indonésia,1 511 712,40 858,1 348 330


A função lambda é uma função anônima (sem nome), em que o parâmetro é designado por celula. A função é aplicada sobre cada célula da coluna. Assim para cada célula, a função replace será executada.

In [41]:
df['Casos'] = df.Casos.apply(lambda celula: celula.replace(" ", ""))
df['Mortes'] = df.Mortes.apply(lambda celula: celula.replace(" ", ""))
df['Curados'] = df.Curados.apply(lambda celula: celula.replace(" ", ""))

In [42]:
df[df['País'] == 'Indonésia']

,País,Casos,Mortes,Curados
20,Indonésia,1511712,40858,1348330


**Convertendo para tipos numéricos**

Se tudo correr bem nessa conversão, é porque tudo foi tratado adequadamente. Caso ocorra problemas aqui, você deve inspecionar a tabela para encontrar onde estão os problemas.

In [45]:
df['Casos'] = df.Casos.astype(int)
df['Mortes'] = df.Mortes.astype(int)
df['Curados'] = df.Curados.astype(int)

### Tarefas

1) Utilizando o plotly, faça a análise da evolução de Casos, Mortes e Curados do Dataset. 

2) Calcule a estatística descritiva (média, soma, mediana e desvio-padrão) dos países do dataset.

3) Encontre uma tabela na Wikipedia do seu interesse e aplique as técnicas apresentadas neste caderno para criar um DataFrame, criar gráficos no Plotly e fazer as estatísticas descritivas.